In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

train_data = pd.read_excel("data/train.xlsx")
test_data = pd.read_excel("data/test.xlsx")
    
print("Первые 5 строк данных:")
print(train_data.head())
print(test_data.head())

print("\nИнформация о данных:")
print(train_data.info())
print(test_data.info())

print("\nОписательная статистика:")
print(train_data.describe())
print(test_data.describe())

print("\nПропущенные значения:")
print(train_data.isnull().sum())
print(test_data.isnull().sum())

print("\nКоличество дубликатов:")
print(train_data.duplicated().sum())
print(test_data.duplicated().sum())

Первые 5 строк данных:
          dt  Price
0 2015-01-05  32900
1 2015-01-12  32500
2 2015-01-19  32000
3 2015-01-26  30700
4 2015-02-02  30300
          dt  Цена на арматуру
0 2022-09-05             46000
1 2022-09-12             46000
2 2022-09-19             46000
3 2022-09-26             45500
4 2022-10-03             45500

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   dt      400 non-null    datetime64[ns]
 1   Price   400 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.4 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dt                28 non-null     datetime64[ns]
 1   Цена на арматуру  28 non-null     int6

In [59]:
for col in test_data.columns:
    pct_missing = np.mean(test_data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

dt - 0%
Цена на арматуру - 0%


In [74]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# 1. Загрузка данных
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 2. Предобработка данных
# Предположим, что целевая переменная — это 'price'
X = train_data.drop(columns=["price", "date"])  # Убираем целевую переменную и дату
y = train_data["price"]

# Если есть категориальные признаки, укажите их индексы
cat_features = list(X.select_dtypes(include=['object']).columns)
cat_indices = [X.columns.get_loc(col) for col in cat_features]

# 3. Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Обучение модели CatBoost
model = CatBoostRegressor(
    iterations=1000,  # Количество итераций
    learning_rate=0.1,  # Скорость обучения
    depth=6,  # Глубина дерева
    cat_features=cat_indices,  # Категориальные признаки
    verbose=100  # Вывод лога каждые 100 итераций
)

model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

# 5. Прогнозирование на валидационной выборке
y_pred = model.predict(X_val)

# 6. Оценка модели
print("MAE:", mean_absolute_error(y_val, y_pred))
print("RMSE:", mean_squared_error(y_val, y_pred, squared=False))

# 7. Прогнозирование на тестовых данных
X_test = test_data.drop(columns=["date"])  # Убираем дату
test_predictions = model.predict(X_test)

# Добавляем прогнозы в тестовые данные
test_data["predicted_price"] = test_predictions

# 8. Принятие решения о закупке
# Пример: если цена растет, закупаем на большее количество недель
def recommend_purchase_weeks(predicted_price, current_price):
    if predicted_price > current_price:
        return 6  # Закупаем на 6 недель вперед
    else:
        return 1  # Закупаем на 1 неделю

test_data["recommended_weeks"] = test_data.apply(
    lambda row: recommend_purchase_weeks(row["predicted_price"], row["price"]), axis=1
)

# 9. Сохранение результатов
test_data.to_csv("test_with_predictions.csv", index=False)

# 10. Сохранение модели
model.save_model("catboost_model.cbm")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject